In [1]:
#General imports and file-reading support.
import os
import sys
import glob
import numpy as np
import time
import shutil

##JustSparkThings.
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql import Row


#sys.path.append('/home/ubuntu/LDSA_group9')
#from hdf5_getters import * 

def dir_explorer(root): 
    file_paths = []
    walker = os.walk(root)
    for (path, dirnames, filenames) in walker:
        for filename in filenames:
            file_paths.append(os.path.join(path, filename))
    return file_paths

def get_title_year_danceability(filename):
    f = open_h5_file_read(filename)
    title = get_title(f)
    duration = get_duration(f)
    year = get_year(f)
    f.close()
    return (title.decode('utf-8'), year, duration)
    
    
conf = SparkConf().setMaster('spark://192.168.1.82:7077').setAppName('Danceability')
sc = SparkContext(conf=conf)
sc.addPyFile("/home/ubuntu/LDSA_group9/hdf5_getters.py")
from hdf5_getters import *

In [2]:
root_directory = "/home/ubuntu/data"
file_paths = dir_explorer(root_directory)

In [3]:
def map_average(entry):
    return (entry[1], (1, entry[2])) 

In [4]:
def reduce_average(a, b):
    count = a[0]+b[0]
    return count, (a[0]*a[1]+b[0]*b[1])/count

In [5]:
start_time = time.time()

In [6]:
filepath_rdd = sc.parallelize(file_paths[0:int(len(file_paths)*(5/5))]).repartition(15).cache()

In [7]:
song_rdd = filepath_rdd.map(get_title_year_danceability).cache()

In [8]:
filtered_song_rdd = song_rdd.filter(lambda x: x[2] != 0.0 and x[1] != 0).cache()

In [9]:
filtered_song_rdd_average_mapped = filtered_song_rdd.map(map_average).cache()

In [10]:
filtered_song_rdd_average_reduced = filtered_song_rdd_average_mapped.reduceByKey(reduce_average).sortBy(keyfunc=lambda x: x[1][1],ascending=False).cache()

In [11]:
if os.path.isdir("/home/ubuntu/results/song_duration.txt"):
    shutil.rmtree("/home/ubuntu/results/song_duration.txt")
filtered_song_rdd_average_reduced.saveAsTextFile("/home/ubuntu/results/song_duration.txt")

In [12]:
#filtered_song_rdd_average_reduced.take(10)

In [13]:
execution_time = time.time()-start_time

In [14]:
sc.stop()

In [15]:
print(execution_time)

196.9419503211975
